# ComCam pointing drift

This notebook calculates the drift of an image due to the known pointing error during the ComCam campaign.

Craig Lage - 06-Feb-25

In [ ]:
import numpy as np
import pickle as pkl
import matplotlib.pyplot as plt
from astropy.time import Time, TimeDelta
from lsst.daf.butler import Butler
import lsst.summit.utils.butlerUtils as butlerUtils
from lsst.summit.utils.utils import dayObsIntToString
from lsst.summit.utils.efdUtils import calcNextDay
from astropy.coordinates import AltAz, ICRS, EarthLocation, Angle, FK5, SkyCoord
import astropy.units as u
from lsst.obs.lsst.translators.lsst import SIMONYI_LOCATION

In [ ]:
# Below is the wavelength center point for the LSST filters:
wavelengths = {'u':3671, 'g':4827, 'r':6223, 'i':7546, 'z':8691, 'y':9712}

def DeltaAltAz (ra, dec, press, hum, temp, wl, time1, time2):
    # This calculates the change in AltAz during an exposure
    # given the RA/Dec
    skyLocation = SkyCoord(ra*u.deg, dec*u.deg)
    wl = wavelengths[filter] * u.angstrom
    altAz1 = AltAz(obstime=time1, location=SIMONYI_LOCATION, pressure=pressure, 
                 temperature=temperature, relative_humidity=hum, obswl=wl)
    altAz2 = AltAz(obstime=time2, location=SIMONYI_LOCATION, pressure=pressure, 
                 temperature=temperature, relative_humidity=hum, obswl=wl)
    obsAltAz1 = skyLocation.transform_to(altAz1)
    obsAltAz2 = skyLocation.transform_to(altAz2)
    # 1 is at the beginning of the exposure, 2 is at the end
    # el, az are the actual values, prime values reflect the pointing model
    # These are all in degrees
    el1 = obsAltAz1.alt.deg
    az1 = obsAltAz1.az.deg
    el2 = obsAltAz2.alt.deg
    az2 = obsAltAz2.az.deg
    [elPrime1, azPrime1] = DeltaAltAz (el1, az1)
    [elPrime2, azPrime2] = DeltaAltAz (el2, az2)
    # Change values are the change from the beginning to the end of the exposure, in arcseconds
    azChange = (az2 - az1) * 3600.0
    elChange = (el2 - el1) * 3600.0
    return [azChange, elChange]



In [ ]:
butler = Butler('/sdf/group/rubin/repo/main', collections=["LSSTComCam/raw/all","LSSTComCam/calib"])
instrument = 'LSSTComCam'


def CalculateDrift(expId):
    md = butler.get('raw.metadata', detector=4, exposure=expId, instrument=instrument)
    filter = md['FILTBAND']
    pressure = md['PRESSURE'] * u.pascal
    temperature = md['AIRTEMP'] * u.Celsius
    hum = md['HUMIDITY']
    time1 = Time(md['MJD-BEG'], format='mjd', scale='tai')
    time2 = Time(md['MJD-END'], format='mjd', scale='tai')
    ra = md['RASTART']
    dec = md['DECSTART']
    skyLocation = SkyCoord(ra*u.deg, dec*u.deg)
    wl = wavelengths[filter] * u.angstrom
    altAz1 = AltAz(obstime=time1, location=SIMONYI_LOCATION, pressure=pressure, 
                 temperature=temperature, relative_humidity=hum, obswl=wl)
    altAz2 = AltAz(obstime=time2, location=SIMONYI_LOCATION, pressure=pressure, 
                 temperature=temperature, relative_humidity=hum, obswl=wl)
    obsAltAz1 = skyLocation.transform_to(altAz1)
    obsAltAz2 = skyLocation.transform_to(altAz2)
    # 1 is at the beginning of the exposure, 2 is at the end
    # el, az are the actual values, prime values reflect the pointing model
    # These are all in degrees
    el1 = obsAltAz1.alt.deg
    az1 = obsAltAz1.az.deg
    el2 = obsAltAz2.alt.deg
    az2 = obsAltAz2.az.deg
    [elPrime1, azPrime1] = DeltaAltAz (el1, az1)
    [elPrime2, azPrime2] = DeltaAltAz (el2, az2)
    # Change values are the change from the beginning to the end of the exposure, in arcseconds
    azChange = (az2 - az1) * 3600.0
    elChange = (el2 - el1) * 3600.0
    azPrimeChange = (azPrime2 - azPrime1) * 3600.0
    elPrimeChange = (elPrime2 - elPrime1) * 3600.0
    azDrift = azChange - azPrimeChange
    elDrift = elChange - elPrimeChange
    print(f" For {expId}, Azimuth drift = {azDrift:.2f} arcseconds, Elevation drift = {elDrift:.2f} arcseconds")
    return

def CalculateDriftMultiple(expId1, expId2):
    md1 = butler.get('raw.metadata', detector=4, exposure=expId1, instrument=instrument)
    md2 = butler.get('raw.metadata', detector=4, exposure=expId2, instrument=instrument)
    filter = md1['FILTBAND']
    pressure = md1['PRESSURE'] * u.pascal
    temperature = md1['AIRTEMP'] * u.Celsius
    hum = md1['HUMIDITY']
    time1 = Time(md1['MJD-BEG'], format='mjd', scale='tai')
    time2 = Time(md2['MJD-END'], format='mjd', scale='tai')
    ra = md1['RASTART']
    dec = md1['DECSTART']
    skyLocation = SkyCoord(ra*u.deg, dec*u.deg)
    wl = wavelengths[filter] * u.angstrom
    altAz1 = AltAz(obstime=time1, location=SIMONYI_LOCATION, pressure=pressure, 
                 temperature=temperature, relative_humidity=hum, obswl=wl)
    altAz2 = AltAz(obstime=time2, location=SIMONYI_LOCATION, pressure=pressure, 
                 temperature=temperature, relative_humidity=hum, obswl=wl)
    obsAltAz1 = skyLocation.transform_to(altAz1)
    obsAltAz2 = skyLocation.transform_to(altAz2)
    # 1 is at the beginning of the exposure, 2 is at the end
    # el, az are the actual values, prime values reflect the pointing model
    # These are all in degrees
    el1 = obsAltAz1.alt.deg
    az1 = obsAltAz1.az.deg
    el2 = obsAltAz2.alt.deg
    az2 = obsAltAz2.az.deg
    [elPrime1, azPrime1] = DeltaAltAz (el1, az1)
    [elPrime2, azPrime2] = DeltaAltAz (el2, az2)
    # Change values are the change from the beginning to the end of the exposure, in arcseconds
    azChange = (az2 - az1) * 3600.0
    elChange = (el2 - el1) * 3600.0
    azPrimeChange = (azPrime2 - azPrime1) * 3600.0
    elPrimeChange = (elPrime2 - elPrime1) * 3600.0
    azDrift = azChange - azPrimeChange
    elDrift = elChange - elPrimeChange
    print(f" For exposures from {expId1} to {expId2}, Azimuth drift = {azDrift:.2f} arcseconds, Elevation drift = {elDrift:.2f} arcseconds")
    return


In [ ]:
CalculateDrift(2024120700551)

In [ ]:
CalculateDrift(2024120800407)

In [ ]:
CalculateDriftMultiple(2024110900229, 2024110900248)

In [ ]:
#butler = butlerUtils.makeDefaultButler("LSSTComCam", embargo=True)
butler = Butler('embargo', collections=["LSSTComCam/raw/all",
                   "LSSTComCam/calib",
                    "LSSTComCam/runs/nightlyValidation/20241109/d_2024_11_05/DM-47059"])
#butler = Butler('/repo/main', collections=["LSSTComCam/raw/all",
#                   "LSSTComCam/calib",
#                    "LSSTComCam/runs/DRP/DP1/w_2025_07/DM-48940"])

instrument = 'LSSTComCam'

In [ ]:
expId = 2024110900229
calExp = butler.get('calexp', detector=4, visit=expId, instrument="LSSTComCam")
rawExp = butler.get('raw', detector=4, exposure=expId, instrument="LSSTComCam")

In [ ]:
# Below is the wavelength center point for the LSST filters:
wavelengths = {'u':3671, 'g':4827, 'r':6223, 'i':7546, 'z':8691, 'y':9712}

def DeltaAltAz (el, az):
    # This calculates the offset due to the pointing model
#    # as extracted from ComCam images.
    deltaEl = -125.4 - 187.9 * np.sin(az * np.pi / 180.0)
    deltaAz = -222.0 - 348.1 * np.cos(az * np.pi / 360.0)
    elPrime = el + deltaEl / 3600.0
    azPrime = az + deltaAz / 3600.0
    return [elPrime, azPrime]

def DeltaAltAz2 (el, az):
    # This calculates the offset due to the pointing model
#    # as extracted from ComCam images.
    deltaEl = 125.4 - 187.9 * np.sin(az * np.pi / 180.0)
    deltaAz = 222.0 + 348.1 * np.cos(az * np.pi / 360.0)
    return [deltaEl, deltaAz]


In [ ]:
rawRas = []
rawDecs = []
rawEls = []
rawAzs = []
calRas = []
calDecs = []
calEls = []
calAzs = []
deltaRas = []
deltaDecs = []
for expId in range(2024110900229, 2024110900249):
    md = butler.get('raw.metadata', detector=4, exposure=expId, instrument=instrument)
    filter = md['FILTBAND']
    pressure = md['PRESSURE'] * u.pascal
    temperature = md['AIRTEMP'] * u.Celsius
    hum = md['HUMIDITY']
    time1 = Time(md['MJD-BEG'], format='mjd', scale='tai')
    time2 = Time(md['MJD-END'], format='mjd', scale='tai')
    time = time1
    ra = md['RASTART']
    dec = md['DECSTART']
    rawSkyLocation = SkyCoord(ra=ra*u.deg, dec=dec*u.deg, frame='icrs')
    wl = wavelengths[filter] * u.angstrom
    altAz = AltAz(obstime=time, location=SIMONYI_LOCATION, pressure=pressure, 
                 temperature=temperature, relative_humidity=hum, obswl=wl)
    rawAltAz = rawSkyLocation.transform_to(altAz)
    rawEl = rawAltAz.alt.deg
    rawAz = rawAltAz.az.deg
    rawEls.append(rawEl)
    rawAzs.append(rawAz)
    #[calEl, calAz] = DeltaAltAz (rawEl, rawAz)

    calExp = butler.get('calexp', detector=4, visit=expId, instrument="LSSTComCam")
    rawExp = butler.get('raw', detector=4, exposure=expId, instrument="LSSTComCam")
    cWcs = calExp.getWcs()
    rWcs = rawExp.getWcs()
    rawSkyCenter = rWcs.getSkyOrigin()
    calExpSkyCenter = cWcs.pixelToSky(rWcs.getPixelOrigin())
    rawRa = rawSkyCenter.getRa().asDegrees()
    rawDec = rawSkyCenter.getDec().asDegrees()
    rawRas.append(rawRa)
    rawDecs.append(rawDec)
    calRa = calExpSkyCenter.getRa().asDegrees()
    calDec = calExpSkyCenter.getDec().asDegrees()
    calSkyLocation = SkyCoord(ra=calRa*u.deg, dec=calDec*u.deg, frame='icrs')
    calAltAz = calSkyLocation.transform_to(altAz)
    calEl = calAltAz.alt.deg
    calAz = calAltAz.az.deg
    calEls.append(calEl)
    calAzs.append(calAz)

    calRas.append(calRa)
    calDecs.append(calDec)
    deltaRa = (rawRa - calRa) * 3600.0
    deltaDec = (rawDec - calDec) * 3600.0
    deltaRas.append(deltaRa)
    deltaDecs.append(deltaDec)
    print(expId, deltaRa, deltaDec)
    deltaEl = (rawEl - calEl) * 3600.0
    deltaAz = (rawAz - calAz) * 3600.0
    [deltaEl2, deltaAz2] = DeltaAltAz2 (rawEl, rawAz)

    print(expId, deltaEl, deltaAz)
    print(expId, deltaEl2, deltaAz2)
    print(rawEl, rawAz)
    print(calEl, calAz)

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(5,5))
ax.plot(deltaRas, color='red')
ax1 = ax.twinx()
ax1.plot(deltaDecs, color='blue')


In [ ]:
fig, axs = plt.subplots(1,2, figsize=(10,5))
axs[0].plot(rawRas)
axs[0].plot(calRas)
axs[1].plot(rawDecs)
axs[1].plot(calDecs)

In [ ]:
fig, axs = plt.subplots(1,2, figsize=(10,5))
axs[0].plot(rawEls)
axs[0].plot(calEls)
axs[1].plot(rawAzs)
axs[1].plot(calAzs)

In [ ]:
print(ra, dec)
print(rawRa, rawDec)
print(calRa, calDec)